A ternary search tree has nodes with the following attributes:
* a character, can be `None`;
* a Boolean flag that indicates whether the character represented
  by this node has been the last in a string that was inserted in the
  tree;
* the "less-than" child;
* the "equals" child and
* the "larger-than" child.

The data structure should support the following operations:
* string insert
* string search
* prefix string search
* return the number of strings stored in the data structure
* return all strings stored in the data structure

Also ensure that an instance of the data structure can be visualy represented, e.g., in aSCII format.

In [1]:
import time
import random
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from typing import List, Tuple

Ternary Search Tree Implementation: \
    *Represents a node in a Ternary Search Tree (TST).\
    *Each node stores a character, a flag indicating if it's the end of a word,\
    *and pointers to left, middle, and right child nodes.

In [2]:
class TSTNode:
    def __init__(self, char):
        self.char = char
        self.is_end = False
        self.left = None
        self.middle = None
        self.right = None

In [3]:

class TernarySearchTree:
    def __init__(self):
        self.root = None

    def insert(self, word):
        if not word:
            return
        self.root = self._insert(self.root, word, 0)

    def _insert(self, node, word, index):
        char = word[index]

        if node is None:
            node = TSTNode(char)

        if char < node.char:
            node.left = self._insert(node.left, word, index)
        elif char > node.char:
            node.right = self._insert(node.right, word, index)
        else:
            if index + 1 < len(word):
                node.middle = self._insert(node.middle, word, index + 1)
            else:
                node.is_end = True
        return node
    def search(self, word):
        if not word:
            return False
        return self._search(self.root, word, 0)

    def _search(self, node, word, index):
        if node is None:
            return False

        char = word[index]

        if char < node.char:
            return self._search(node.left, word, index)
        elif char > node.char:
            return self._search(node.right, word, index)
        else:
            if index == len(word) - 1:
                return node.is_end
            return self._search(node.middle, word, index + 1)

In [ ]:
def main():

    # === Define File Paths ===
    insert_words_path = Path("D:/Second Semester/Concept of Data Science/Project/data/data/search_trees/insert_words.txt")
    not_insert_words_path = Path("D:/Second Semester/Concept of Data Science/Project/data/data/search_trees/not_insert_words.txt")
    corncob_path = Path("D:/Second Semester/Concept of Data Science/Project/data/data/search_trees/corncob_lowercase.txt")

    # === Load Words from Files ===
    try:
        with open(insert_words_path, "r", encoding="utf-8") as f:
            insert_words = [line.strip() for line in f if line.strip()]
        with open(not_insert_words_path, "r", encoding="utf-8") as f:
            not_insert_words = [line.strip() for line in f if line.strip()]
        with open(corncob_path, "r", encoding="utf-8") as f:
            large_word_list = [line.strip() for line in f if line.strip()]
    except FileNotFoundError as e:
        print(f"Error loading words: {e}. Make sure that the file location is correct.")
        print(f"Expected path for insert_words.txt: {insert_words_path}")
        print(f"Expected path for not_insert_words.txt: {not_insert_words_path}")
        print(f"Expected path for corncob_lowercase.txt: {corncob_path}")
        return
    
    # === Build and Test Tree for Correctness ===
    print("--- Correctness Testing ---")
    tst = TernarySearchTree()
    print(f"Inserting {len(insert_words)} words into the TST...")
    for word in insert_words:
        tst.insert(word)
    print("Insertion complete.")

    print("\nVerifying Inserted Words (Expected True):")
    all_correct_inserted = True
    for word in insert_words:
        result = tst.search(word)
        print(f"'{word}': {result}")
        if not result:
            all_correct_inserted = False
    if all_correct_inserted:
        print("All inserted words found correctly.")
    else:
        print("Warning: Some inserted words were not found or not marked as end of word.")